In [30]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.core import Var
import numpy as np

In [41]:
lmda = 21.97 #cent/kWh retail electricity price
pl = [8,8,10,10,10,16,22,24,26,32,30,28,22,18,16,16,20,24,28,34,38,30,22,12] #load values

# fuel cost parameters
c2 = np.array([1.2,1.12])*0.001
c1 = np.array([0.128,0.532])
c  = np.array([2.12,12.8])*0.0001

# min max power values
pmin = np.array([0,0])
pmax = np.array([20,40])

In [38]:
# Dont properly understand between Concrete and Abstract model
model = pyo.ConcreteModel()

#Hour Set
model.H = pyo.RangeSet(0,23)

#Generator Set
model.G = pyo.RangeSet(0,1)

#Variables
model.pn = pyo.Var(model.H) #Net power needed from external network dependent on Hourly Set
model.pg = pyo.Var(model.H,model.G, within=pyo.NonNegativeReals) #Power Generation of each Generator per Hour
model.u = pyo.Var(model.H,model.G, within=pyo.Binary) # Unit commitment Variable for each Generator and Hour


#Objective Function: First Part is Net power cost with DisCo, Second part is fuel costs
def obj_rule(model):
    return sum(lmda*model.pn[h] for h in model.H) + sum(((c2[g]*model.pg[h,g] + c1[g]*model.pg[h,g] + c[g])*model.u[h,g] for h in model.H for g in model.G))
model.OBJ = pyo.Objective(rule=obj_rule)


#Load Constraint
def loadc(model,H):
    return sum(model.pg[H,g] for g in model.G) + model.pn[H] == pl[H]
model.loadc = pyo.Constraint(model.H, rule = loadc)

#Min-Constraint
def minc(model,H,G):
    return model.u[H,G]*pmin[G] <= model.pg[H,G] 
model.minc = pyo.Constraint(model.H,model.G, rule = minc)

#Max-Constraint
def maxc(model,H,G):
    return model.u[H,G]*pmax[G] >= model.pg[H,G] 
model.maxc = pyo.Constraint(model.H,model.G, rule = maxc)


#SolverFactory('gurobi').solve(model).write()
opt = SolverFactory('gurobi')
results = opt.solve(model, tee=True)
results.write()
model.solutions.load_from(results)

Using license file /Users/Eric/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/_d/mv3l2b350l32gxgk9ndhl2ww0000gn/T/tmpttsd8b26.pyomo.lp
Reading time = 0.00 seconds
x121: 121 rows, 121 columns, 217 nonzeros
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 121 rows, 121 columns and 217 nonzeros
Model fingerprint: 0xb40faf7f
Model has 48 quadratic objective terms
Variable types: 73 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e-04, 2e+01]
  QObjective range [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 11072.915808
Presolve removed 118 rows and 117 columns
Presolve time: 0.01s
Presolved: 5 rows, 6 columns, 13 nonzeros
Found heuristic solution: objective 713.8415680
Variable types: 4 continuous, 2 integer (2 binary)

Root relaxation: objective 1.055305e+02, 3 iterations

In [39]:
for v in model.component_objects(Var, active=True):
    print ("Variable", v)
    varobject = getattr(model, str(v))
    for index in varobject:
        print ("   ",index, varobject[index].value)

Variable pn
    0 0.0
    1 0.0
    2 0.0
    3 0.0
    4 0.0
    5 0.0
    6 0.0
    7 0.0
    8 0.0
    9 0.0
    10 0.0
    11 0.0
    12 0.0
    13 0.0
    14 0.0
    15 0.0
    16 0.0
    17 0.0
    18 0.0
    19 0.0
    20 0.0
    21 0.0
    22 0.0
    23 0.0
Variable pg
    (0, 0) 8.0
    (0, 1) 0.0
    (1, 0) 8.0
    (1, 1) 0.0
    (2, 0) 10.0
    (2, 1) 0.0
    (3, 0) 10.0
    (3, 1) 0.0
    (4, 0) 10.0
    (4, 1) 0.0
    (5, 0) 16.0
    (5, 1) 0.0
    (6, 0) 20.0
    (6, 1) 2.0
    (7, 0) 20.0
    (7, 1) 4.0
    (8, 0) 20.0
    (8, 1) 6.0
    (9, 0) 20.0
    (9, 1) 12.0
    (10, 0) 20.0
    (10, 1) 10.0
    (11, 0) 20.0
    (11, 1) 8.0
    (12, 0) 20.0
    (12, 1) 2.0
    (13, 0) 18.0
    (13, 1) 0.0
    (14, 0) 16.0
    (14, 1) 0.0
    (15, 0) 16.0
    (15, 1) 0.0
    (16, 0) 20.0
    (16, 1) 0.0
    (17, 0) 20.0
    (17, 1) 4.0
    (18, 0) 20.0
    (18, 1) 8.0
    (19, 0) 20.0
    (19, 1) 14.0
    (20, 0) 20.0
    (20, 1) 18.0
    (21, 0) 20.0
    (21, 1) 10.0
    (22, 0) 2